In [1]:

from utils import *



%matplotlib inline
data_path = 'data/'
seed=1204

submission_path=data_path+'submission/'
fold_path = 'fold_data/'


cv_loss_list=[]
n_iteration_list=[]
def score(params):
    print("Training with params: ")
    print(params)
    cv_losses=[]
    cv_iteration=[]
    for (train_idx,val_idx) in cv:
        cv_train = X.iloc[train_idx]
        cv_val = X.iloc[val_idx]
        cv_y_train = y[train_idx]
        cv_y_val = y[val_idx]
        
        dtrain = xgb.DMatrix(cv_train,cv_y_train)
        dval = xgb.DMatrix(cv_val,cv_y_val)
        watchlist = [(dtrain, 'train'), (dval, 'valid')]
        
        xgb_model = xgb.train(params, dtrain, 2000, watchlist,
                          verbose_eval=False, 
                          early_stopping_rounds=200)
       
        train_pred = xgb_model.predict(dtrain,ntree_limit=xgb_model.best_ntree_limit)
        val_pred = xgb_model.predict(dval,ntree_limit=xgb_model.best_ntree_limit+1)
        train_loss = root_mean_squared_error(cv_y_train,train_pred)
        val_loss = root_mean_squared_error(cv_y_val,val_pred)
        print('Train RMSE: {},Val RMSE: {}'.format(train_loss,val_loss))
        print('Best iteration: {}'.format(xgb_model.best_ntree_limit))
        cv_losses.append(val_loss)
        cv_iteration.append(xgb_model.best_iteration)
        
        xgb_model.__del__()
    print('6 fold results: {}'.format(cv_losses))
    
    cv_loss_list.append(cv_losses)
    n_iteration_list.append(cv_iteration)
    
    mean_cv_loss = np.mean(cv_losses)
    print('Average iterations: {}'.format(np.mean(cv_iteration)))
    print("Mean Cross Validation RMSE: {}\n".format(mean_cv_loss))
    return {'loss': mean_cv_loss, 'status': STATUS_OK}

def optimize(space,seed=seed,max_evals=5):
    
    best = fmin(score, space, algo=tpe.suggest, 
        # trials=trials, 
        max_evals=max_evals)
    return best


In [2]:
all_data = get_all_data(data_path,'new_sales_lag_after12.pickle')

X,y = get_X_y(all_data,33)
X.drop('date_block_num',axis=1,inplace=True)

cv = get_cv_idxs(all_data,28,33)

# vanila_params = {'eval_metric': 'rmse','objective': "reg:linear" , 'max_depth':11,'seed': 1204,'tree_method':'gpu_hist'}

# Hyperopt

In [2]:
all_data = get_all_data(data_path,'new_sales_lag_after12_month.pickle')

X,y = get_X_y(all_data,33)
X.drop('date_block_num',axis=1,inplace=True)

cv = get_cv_idxs(all_data,28,33)


In [3]:
space = {
    #'n_estimators': hp.quniform('n_estimators', 50, 500, 5),
#     'max_depth': hp.choice('max_depth', np.arange(5, 10, dtype=int)),
    'subsample': hp.quniform('subsample', 0.7, 0.9, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.7, 0.9, 0.05),
    'gamma': hp.quniform('gamma', 0, 1, 0.05),
    'max_leaf_nodes': hp.choice('max_leaf_nodes', np.arange(100,140, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(100,140, dtype=int)),
    'learning_rate': 0.03,
    'eval_metric': 'rmse',
    'objective': 'reg:linear' , 
    'seed': 1204,'tree_method':'gpu_hist'
}
best_hyperparams = optimize(space,max_evals=200)
print("The best hyperparameters are: ")
print(best_hyperparams)

Training with params: 
{'colsample_bytree': 0.8500000000000001, 'eval_metric': 'rmse', 'gamma': 0.05, 'learning_rate': 0.03, 'max_leaf_nodes': 126, 'min_child_weight': 139, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.9, 'tree_method': 'gpu_hist'}
Train RMSE: 0.7692943150150477,Val RMSE: 0.8298628584037769
Best iteration: 1586
Train RMSE: 0.8150147987262512,Val RMSE: 0.7748194976537419
Best iteration: 504
Train RMSE: 0.7595488722059013,Val RMSE: 0.7014695069801841
Best iteration: 1985
Train RMSE: 0.7555471559093475,Val RMSE: 0.7630134729718234
Best iteration: 2000
Train RMSE: 0.7586298595127628,Val RMSE: 0.878838464454166
Best iteration: 1861
Train RMSE: 0.865694702867586,Val RMSE: 0.9411882330092844
Best iteration: 98
6 fold results: [0.8298628584037769, 0.7748194976537419, 0.7014695069801841, 0.7630134729718234, 0.878838464454166, 0.9411882330092844]
Average iterations: 1338.0
Mean Cross Validation RMSE: 0.8148653389121628

Training with params: 
{'colsample_bytree': 0.850

Train RMSE: 0.7820473988453849,Val RMSE: 0.8812068076404532
Best iteration: 1050
Train RMSE: 0.8499738941671597,Val RMSE: 0.9340865937204799
Best iteration: 154
6 fold results: [0.8304538370194816, 0.775360837119202, 0.6990630759730074, 0.7633207640869342, 0.8812068076404532, 0.9340865937204799]
Average iterations: 1238.0
Mean Cross Validation RMSE: 0.8139153192599266

Training with params: 
{'colsample_bytree': 0.9, 'eval_metric': 'rmse', 'gamma': 0.25, 'learning_rate': 0.03, 'max_leaf_nodes': 103, 'min_child_weight': 129, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.75, 'tree_method': 'gpu_hist'}
Train RMSE: 0.7689951079477138,Val RMSE: 0.8338287708641317
Best iteration: 1958
Train RMSE: 0.8101831562279217,Val RMSE: 0.7762536691649575
Best iteration: 619
Train RMSE: 0.7699212572162072,Val RMSE: 0.7009607491313181
Best iteration: 1835
Train RMSE: 0.7632987825689573,Val RMSE: 0.7671243816445966
Best iteration: 1999
Train RMSE: 0.7936126650249558,Val RMSE: 0.8850010101937601


Train RMSE: 0.7786129888201233,Val RMSE: 0.8328703110545863
Best iteration: 1390
Train RMSE: 0.8005282490837061,Val RMSE: 0.7727447644207214
Best iteration: 763
Train RMSE: 0.7720638754489715,Val RMSE: 0.7020154675888078
Best iteration: 1563
Train RMSE: 0.7721313853852835,Val RMSE: 0.7658928383203124
Best iteration: 1462
Train RMSE: 0.7779602213352039,Val RMSE: 0.8842784222280834
Best iteration: 1191
Train RMSE: 0.869065219308384,Val RMSE: 0.9402616198165826
Best iteration: 90
6 fold results: [0.8328703110545863, 0.7727447644207214, 0.7020154675888078, 0.7658928383203124, 0.8842784222280834, 0.9402616198165826]
Average iterations: 1075.5
Mean Cross Validation RMSE: 0.816343903904849

Training with params: 
{'colsample_bytree': 0.7000000000000001, 'eval_metric': 'rmse', 'gamma': 0.75, 'learning_rate': 0.03, 'max_leaf_nodes': 139, 'min_child_weight': 136, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.8, 'tree_method': 'gpu_hist'}
Train RMSE: 0.7696735618404156,Val RMSE: 0.83136

Train RMSE: 0.7753412341474909,Val RMSE: 0.8334675958875802
Best iteration: 1665
Train RMSE: 0.7965870131158769,Val RMSE: 0.7710846070401707
Best iteration: 856
Train RMSE: 0.7821053590824806,Val RMSE: 0.7028347475933053
Best iteration: 1254
Train RMSE: 0.7686692652913436,Val RMSE: 0.7647028059649402
Best iteration: 1788
Train RMSE: 0.7742598814292161,Val RMSE: 0.8813715627709033
Best iteration: 1456
Train RMSE: 0.8408162009463208,Val RMSE: 0.9402300819898033
Best iteration: 194
6 fold results: [0.8334675958875802, 0.7710846070401707, 0.7028347475933053, 0.7647028059649402, 0.8813715627709033, 0.9402300819898033]
Average iterations: 1201.1666666666667
Mean Cross Validation RMSE: 0.8156152335411172

Training with params: 
{'colsample_bytree': 0.8, 'eval_metric': 'rmse', 'gamma': 0.5, 'learning_rate': 0.03, 'max_leaf_nodes': 107, 'min_child_weight': 104, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.75, 'tree_method': 'gpu_hist'}
Train RMSE: 0.7920756831059718,Val RMSE: 0.83919

KeyboardInterrupt: 

In [3]:
# space = {
#     #'n_estimators': hp.quniform('n_estimators', 50, 500, 5),
# #     'max_depth': hp.choice('max_depth', np.arange(5, 10, dtype=int)),
#     'subsample': hp.quniform('subsample', 0.7, 0.9, 0.05),
#     'colsample_bytree': hp.quniform('colsample_bytree', 0.7, 0.9, 0.05),
#     'min_child_weight': 128,
#     'learning_rate': 0.03,
#     'eval_metric': 'rmse',
#     'objective': 'reg:linear' , 
#     'seed': 1204,'tree_method':'gpu_hist'
# }
# best_hyperparams = optimize(space,max_evals=200)
# print("The best hyperparameters are: ")
# print(best_hyperparams)


Training with params: 
{'colsample_bytree': 0.8, 'eval_metric': 'rmse', 'learning_rate': 0.03, 'min_child_weight': 128, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.75, 'tree_method': 'gpu_hist'}
Train RMSE: 0.7965656876977267,Val RMSE: 0.8381787496082568
Best iteration: 875
Train RMSE: 0.8181629172704371,Val RMSE: 0.7767925512705117
Best iteration: 481
Train RMSE: 0.7665396306964919,Val RMSE: 0.6994470830865399
Best iteration: 1995
Train RMSE: 0.763078971618149,Val RMSE: 0.7622859701350537
Best iteration: 1999
Train RMSE: 0.7889513749536918,Val RMSE: 0.8840298656536524
Best iteration: 901
Train RMSE: 0.8530734245723749,Val RMSE: 0.9329739672084194
Best iteration: 139
6 fold results: [0.8381787496082568, 0.7767925512705117, 0.6994470830865399, 0.7622859701350537, 0.8840298656536524, 0.9329739672084194]
Average iterations: 1064.0
Mean Cross Validation RMSE: 0.8156180311604055

Training with params: 
{'colsample_bytree': 0.8, 'eval_metric': 'rmse', 'learning_rate': 0.03, 'min_

Train RMSE: 0.7759319116865128,Val RMSE: 0.8320097226505544
Best iteration: 1451
Train RMSE: 0.8026895626581902,Val RMSE: 0.7749414944869222
Best iteration: 714
Train RMSE: 0.7623087627547279,Val RMSE: 0.6989193067348048
Best iteration: 1958
Train RMSE: 0.7700826924991517,Val RMSE: 0.76431676460316
Best iteration: 1494
Train RMSE: 0.7752535143873757,Val RMSE: 0.8770550029344238
Best iteration: 1281
Train RMSE: 0.8644767182272546,Val RMSE: 0.9350729679765752
Best iteration: 99
6 fold results: [0.8320097226505544, 0.7749414944869222, 0.6989193067348048, 0.76431676460316, 0.8770550029344238, 0.9350729679765752]
Average iterations: 1165.1666666666667
Mean Cross Validation RMSE: 0.8137192098977399

Training with params: 
{'colsample_bytree': 0.7000000000000001, 'eval_metric': 'rmse', 'learning_rate': 0.03, 'min_child_weight': 128, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.75, 'tree_method': 'gpu_hist'}
Train RMSE: 0.78970012773955,Val RMSE: 0.8345347248027192
Best iteration: 1

Train RMSE: 0.7666825367141921,Val RMSE: 0.8311973655500142
Best iteration: 1948
Train RMSE: 0.8185005885537466,Val RMSE: 0.7753848981285482
Best iteration: 480
Train RMSE: 0.7713644200033555,Val RMSE: 0.699542711107383
Best iteration: 1675
Train RMSE: 0.7606152096747527,Val RMSE: 0.7641360538081902
Best iteration: 1994
Train RMSE: 0.7606396587655077,Val RMSE: 0.876602714554476
Best iteration: 2000
Train RMSE: 0.8547715433423312,Val RMSE: 0.9345408148978462
Best iteration: 133
6 fold results: [0.8311973655500142, 0.7753848981285482, 0.699542711107383, 0.7641360538081902, 0.876602714554476, 0.9345408148978462]
Average iterations: 1370.6666666666667
Mean Cross Validation RMSE: 0.8135674263410763

Training with params: 
{'colsample_bytree': 0.8500000000000001, 'eval_metric': 'rmse', 'learning_rate': 0.03, 'min_child_weight': 128, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.8, 'tree_method': 'gpu_hist'}
Train RMSE: 0.7735555924705648,Val RMSE: 0.8292124464026535
Best iteration:

Train RMSE: 0.7900460411489715,Val RMSE: 0.8314961965133284
Best iteration: 1001


KeyboardInterrupt: 

In [5]:
# space = {
#     #'n_estimators': hp.quniform('n_estimators', 50, 500, 5),
#     'max_depth': hp.choice('max_depth', np.arange(5, 10, dtype=int)),
#     'subsample': hp.quniform('subsample', 0.7, 1, 0.1),
#     'colsample_bytree': hp.quniform('colsample_bytree', 0.7, 1, 0.1),
#     'min_child_weight': hp.choice('min_child_weight',np.arange(1,17,1, dtype=int)),
#     'learning_rate': hp.quniform('learning_rate', 0.1, 0.3, 0.05),
#     'eval_metric': 'rmse',
#     'objective': 'reg:linear' , 
#     'seed': 1204,'tree_method':'gpu_hist'
# }
# best_hyperparams = optimize(space,max_evals=200)
# print("The best hyperparameters are: ")
# print(best_hyperparams)


Training with params: 
{'colsample_bytree': 0.9, 'eval_metric': 'rmse', 'learning_rate': 0.2, 'max_depth': 8, 'min_child_weight': 14, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 1.0, 'tree_method': 'gpu_hist'}
Train RMSE: 0.7582230782966445,Val RMSE: 0.8403623177602558
Best iteration: 73
Train RMSE: 0.7959162985078891,Val RMSE: 0.7740933498600058
Best iteration: 33
Train RMSE: 0.7423294734686047,Val RMSE: 0.7076492559214119
Best iteration: 82
Train RMSE: 0.7278143240421381,Val RMSE: 0.7711094198596203
Best iteration: 106
Train RMSE: 0.7364223430064114,Val RMSE: 0.9060710204079657
Best iteration: 94
Train RMSE: 0.8119570311701314,Val RMSE: 0.9139115298544712
Best iteration: 21
6 fold results: [0.8403623177602558, 0.7740933498600058, 0.7076492559214119, 0.7711094198596203, 0.9060710204079657, 0.9139115298544712]
Mean Cross Validation RMSE: 0.8188661489439553

Training with params: 
{'colsample_bytree': 0.9, 'eval_metric': 'rmse', 'learning_rate': 0.30000000000000004, 'max_depth

Train RMSE: 0.7595040231363349,Val RMSE: 0.842674488257171
Best iteration: 31
Train RMSE: 0.8183653838089552,Val RMSE: 0.7895589720726386
Best iteration: 10
Train RMSE: 0.7598488573314129,Val RMSE: 0.711099878568482
Best iteration: 31
Train RMSE: 0.706029741155187,Val RMSE: 0.7729019468676093
Best iteration: 72
Train RMSE: 0.6913824562245896,Val RMSE: 0.8845160934532448
Best iteration: 96
Train RMSE: 0.7763840007760797,Val RMSE: 0.9250501490221336
Best iteration: 23
6 fold results: [0.842674488257171, 0.7895589720726386, 0.711099878568482, 0.7729019468676093, 0.8845160934532448, 0.9250501490221336]
Mean Cross Validation RMSE: 0.8209669213735467

Training with params: 
{'colsample_bytree': 0.8, 'eval_metric': 'rmse', 'learning_rate': 0.15000000000000002, 'max_depth': 6, 'min_child_weight': 2, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.9, 'tree_method': 'gpu_hist'}
Train RMSE: 0.7582270088345027,Val RMSE: 0.8440419151284569
Best iteration: 277
Train RMSE: 0.8157996885812983,

KeyboardInterrupt: 

In [ ]:
# space = {
#     #'n_estimators': hp.quniform('n_estimators', 50, 500, 5),
#     'max_depth': hp.choice('max_depth', np.arange(3, 15, dtype=int)),
#     'subsample': hp.quniform('subsample', 0.7, 1, 0.05),
#     'colsample_bytree': hp.quniform('colsample_bytree', 0.7, 1, 0.05),
#     'min_child_weight': hp.choice('min_child_weight',np.arange(1,20,1, dtype=int)),
#     'learning_rate': hp.quniform('learning_rate', 0.1, 0.4, 0.05),
#     'eval_metric': 'rmse',
#     'objective': 'reg:linear' , 
#     'seed': 1204,'tree_method':'gpu_hist'
# }
# best_hyperparams = optimize(space,max_evals=200)
# print("The best hyperparameters are: ")
# print(best_hyperparams)


Training with params: 
{'colsample_bytree': 0.9, 'eval_metric': 'rmse', 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 3, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.9500000000000001, 'tree_method': 'gpu_hist'}
Train RMSE: 0.8104449104611019,Val RMSE: 0.8348437846128616
Best iteration: 136
Train RMSE: 0.7775579186237014,Val RMSE: 0.7757464152310767
Best iteration: 274
Train RMSE: 0.7779337498594463,Val RMSE: 0.7051185321588994
Best iteration: 257
Train RMSE: 0.7262385476763841,Val RMSE: 0.7640047250446783
Best iteration: 652
Train RMSE: 0.7100954623586775,Val RMSE: 0.8654629166546479
Best iteration: 924
Train RMSE: 0.8471344427241345,Val RMSE: 0.9305673811210164
Best iteration: 45
6 fold results: [0.8348437846128616, 0.7757464152310767, 0.7051185321588994, 0.7640047250446783, 0.8654629166546479, 0.9305673811210164]
Mean Cross Validation RMSE: 0.8126239591371968

Training with params: 
{'colsample_bytree': 0.75, 'eval_metric': 'rmse', 'learning_rate': 0.2, 'max_de

Train RMSE: 0.7950262052787478,Val RMSE: 0.844506489746291
Best iteration: 131
Train RMSE: 0.8659788764861783,Val RMSE: 0.7988794958483234
Best iteration: 19
Train RMSE: 0.7782046664103704,Val RMSE: 0.7151547781937673
Best iteration: 171
Train RMSE: 0.7974259584908526,Val RMSE: 0.7711188887157481
Best iteration: 108
Train RMSE: 0.7752094201197631,Val RMSE: 0.8976584996393632
Best iteration: 185
Train RMSE: 0.9188485008428039,Val RMSE: 0.9889926385538049
Best iteration: 4
6 fold results: [0.844506489746291, 0.7988794958483234, 0.7151547781937673, 0.7711188887157481, 0.8976584996393632, 0.9889926385538049]
Mean Cross Validation RMSE: 0.8360517984495496

Training with params: 
{'colsample_bytree': 0.75, 'eval_metric': 'rmse', 'learning_rate': 0.30000000000000004, 'max_depth': 7, 'min_child_weight': 4, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.7000000000000001, 'tree_method': 'gpu_hist'}
Train RMSE: 0.862410818547969,Val RMSE: 0.851106504089474
Best iteration: 8
Train RMSE: 0

# Get oof prediction + test prediction
# Method: Grow tree by tuning max depth

In [8]:
all_data = get_all_data(data_path,'new_sales_lag_after12.pickle')

X,y = get_X_y(all_data,33)
X.drop('date_block_num',axis=1,inplace=True)

cv = get_cv_idxs(all_data,28,33)

In [9]:
xgb_params1={'colsample_bytree': 0.9, 'eval_metric': 'rmse', 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 3, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.9500000000000001, 'tree_method': 'gpu_hist'}

In [7]:
oof_train,_=timeseries_cv('xgb',X,y,xgb_params1,cv,root_mean_squared_error,300,True)
oof_df = pd.Series(oof_train)
oof_df.to_pickle(data_path+'oof/xgb3.pickle')

Training with params: 
{'colsample_bytree': 0.9, 'eval_metric': 'rmse', 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 3, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.9500000000000001, 'tree_method': 'gpu_hist'}
Train RMSE: 0.7639927884929891. Val RMSE: 0.8402815985964945
Best iteration: 366
Train RMSE: 0.7727458057246543. Val RMSE: 0.7779882624949396
Best iteration: 293
Train RMSE: 0.7593112165248824. Val RMSE: 0.7016614941801245
Best iteration: 365
Train RMSE: 0.7544848341735331. Val RMSE: 0.7671823427605784
Best iteration: 392
Train RMSE: 0.707509000606371. Val RMSE: 0.8822958793374427
Best iteration: 993
Train RMSE: 0.7987938759324034. Val RMSE: 0.9338645063609026
Best iteration: 154
n validation fold results: [0.8402815985964945, 0.7779882624949396, 0.7016614941801245, 0.7671823427605784, 0.8822958793374427, 0.9338645063609026]
Average iterations: 427
Mean Cross Validation RMSE: 0.8172123472884136



In [7]:
%%time
dtrain = xgb.DMatrix(X,y)
xgb_full = xgb.train(xgb_params1, dtrain,381*2, [(dtrain, 'train'), (dtrain, 'valid')],
                      verbose_eval=10)

[0]	train-rmse:1.18035	valid-rmse:1.18035
[10]	train-rmse:1.01782	valid-rmse:1.01782
[20]	train-rmse:0.941191	valid-rmse:0.941191
[30]	train-rmse:0.904478	valid-rmse:0.904478
[40]	train-rmse:0.885363	valid-rmse:0.885363
[50]	train-rmse:0.874585	valid-rmse:0.874585
[60]	train-rmse:0.865649	valid-rmse:0.865649
[70]	train-rmse:0.858993	valid-rmse:0.858993
[80]	train-rmse:0.854599	valid-rmse:0.854599
[90]	train-rmse:0.851668	valid-rmse:0.851668
[100]	train-rmse:0.847219	valid-rmse:0.847219
[110]	train-rmse:0.843129	valid-rmse:0.843129
[120]	train-rmse:0.839317	valid-rmse:0.839317
[130]	train-rmse:0.836417	valid-rmse:0.836417
[140]	train-rmse:0.833421	valid-rmse:0.833421
[150]	train-rmse:0.830579	valid-rmse:0.830579
[160]	train-rmse:0.827843	valid-rmse:0.827843
[170]	train-rmse:0.825729	valid-rmse:0.825729
[180]	train-rmse:0.823402	valid-rmse:0.823402
[190]	train-rmse:0.821494	valid-rmse:0.821494
[200]	train-rmse:0.819402	valid-rmse:0.819402
[210]	train-rmse:0.81683	valid-rmse:0.81683
[220]

In [8]:
test = pd.read_csv(os.path.join(data_path, 'test_lag.csv'))

test.drop(['ID','item_name','date_block_num'],axis=1,inplace=True)
dtest= xgb.DMatrix(test)
test_pred = xgb_full.predict(dtest,381*2)
get_submission(test_pred,'tuned_xgb_basicfeatures_6folds_8126');
xgb_full.__del__()

,ID,item_cnt_month
0,0,0.635656
1,1,0.277100
2,2,1.193923
3,3,0.362728
4,4,2.800084
5,5,0.511515
6,6,0.471175
7,7,0.187598
8,8,0.877072
9,9,0.254353


# Get oof prediction + test prediction

# Grow tree max_leaf_node instead of max depth

In [5]:
all_data = get_all_data(data_path,'new_sales_lag_after12.pickle')

X,y = get_X_y(all_data,33)
X.drop('date_block_num',axis=1,inplace=True)

cv = get_cv_idxs(all_data,28,33)
xgb_params2={'colsample_bytree': 0.8500000000000001, 'eval_metric': 'rmse', 'gamma': 0.6000000000000001, 'learning_rate': 0.03, 'max_leaf_nodes': 113, 'min_child_weight': 122, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.75, 'tree_method': 'gpu_hist'}


In [6]:
oof_train,_=timeseries_cv('xgb',X,y,xgb_params2,cv,root_mean_squared_error,200,True)
oof_df = pd.Series(oof_train)
oof_df.to_pickle(data_path+'oof/xgb2.pickle')

Training with params: 
{'colsample_bytree': 0.8500000000000001, 'eval_metric': 'rmse', 'gamma': 0.6000000000000001, 'learning_rate': 0.03, 'max_leaf_nodes': 113, 'min_child_weight': 122, 'objective': 'reg:linear', 'seed': 1204, 'subsample': 0.75, 'tree_method': 'gpu_hist'}
Train RMSE: 0.7855304187054425. Val RMSE: 0.8357879290953042
Best iteration: 1217
Train RMSE: 0.8043950114098816. Val RMSE: 0.7744203738019145
Best iteration: 727
Train RMSE: 0.7686397209773999. Val RMSE: 0.7013283345129842
Best iteration: 1904
Train RMSE: 0.7641108195310308. Val RMSE: 0.7641784470779217
Best iteration: 1962
Train RMSE: 0.7842155628857831. Val RMSE: 0.8876253106745638
Best iteration: 1041
Train RMSE: 0.8177810847159686. Val RMSE: 0.9364247831983158
Best iteration: 404
n validation fold results: [0.8357879290953042, 0.7744203738019145, 0.7013283345129842, 0.7641784470779217, 0.8876253106745638, 0.9364247831983158]
Average iterations: 1209
Mean Cross Validation RMSE: 0.8166275297268341



In [9]:

%%time
dtrain = xgb.DMatrix(X,y)
xgb_full = xgb.train(xgb_params2, dtrain,1109, [(dtrain, 'train'), (dtrain, 'valid')],
                      verbose_eval=10)

test = pd.read_csv(os.path.join(data_path, 'test_lag.csv'))

test.drop(['ID','item_name','date_block_num'],axis=1,inplace=True)
dtest= xgb.DMatrix(test)
test_pred = xgb_full.predict(dtest,1109)
get_submission(test_pred,'tuned_xgb_basicfeatures_6folds_8136');
xgb_full.__del__()